In [3]:
%%capture
# Download dependcies
%pip install -q jmd_imagescraper
%pip install tensorflow_datasets==4.7
%pip install os

If the imports below does not work, you may need to restart the kernel. This can be done in top right "RESTART KERNEL" button.

In [4]:
%%capture
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import Sequential, losses, metrics, layers
import numpy as np
import os
from PIL import Image

The goal in this chapter is to create our own classification model with images sourced from the web. In order to do that, 
here are the necessary steps: 
1. Get images for the web
2. Process images
3. Create dataset with features and labels
4. Create model
5. Train model
6. Evaluate model's performance

1. Get images from the web

Imagine you were to go on google, search for an image, and then copy and paste that image to form your dataset. That's what 
we're doing here, but with code! We're using a module called jmd_imagescraper (which we downloaded and imported in the first 
two cells). This module usees duckduckgo to search for images and download them to our computer. Don't worry about the first
few lines of code in the block below. They're not important to understanding how to train the model. 

In the block below, we downloaded a bunch of images to our computer (see in the project files, you may need to click the file 
refresh button for it to appear) and stored the file names inside of a list called cat_images and dog_images. 

In [5]:
from pathlib import Path
root = Path().cwd()/"images"

from jmd_imagescraper.core import * # dont't worry, it's designed to work with import *

cat_images = duckduckgo_search(root, "Cats", "cute kittens", max_results=100)
dog_images = duckduckgo_search(root, "Dogs", "cute puppies", max_results=100)

Duckduckgo search: cute kittens


Duckduckgo search: cute puppies


In [11]:
print('amount of cat images: ', len(cat_images))
print('amount of dog images: ', len(cat_images))

print('\nfirst 5 cat image file names:')
print(cat_images[:5])

print('\nirst 5 dog image file names:')
print(dog_images[:5])


amount of cat images:  100
amount of dog images:  100

first 5 cat image file names:
[PosixPath('/notebooks/images/Cats/511_5b8babd3.jpg'), PosixPath('/notebooks/images/Cats/512_11ce4387.jpg'), PosixPath('/notebooks/images/Cats/513_e1d7586a.jpg'), PosixPath('/notebooks/images/Cats/514_867fc83f.jpg'), PosixPath('/notebooks/images/Cats/515_611afe9f.jpg')]

irst 5 dog image file names:
[PosixPath('/notebooks/images/Dogs/511_86a8570f.jpg'), PosixPath('/notebooks/images/Dogs/512_da65c7b5.jpg'), PosixPath('/notebooks/images/Dogs/513_bbdfd40e.jpg'), PosixPath('/notebooks/images/Dogs/514_ae8a3b09.jpg'), PosixPath('/notebooks/images/Dogs/515_5b159bda.jpg')]


In the codeblock above, we take a look at cat_images and dog_images. They both contain the file paths to where the images are 
stored in our project. 


Task: 
Can you figure out when there are 100 images? How about trying to change this number? Experiment with getting differnt amount of 
images, and different types of images. Try to get a bunch of different animals!

2. Process Images

Now that we have the images downloaded to our project, we have to prepare them so we can put them in a dataset. Since we're using 
supervised learning, our dataset needs to have both features and labels. Our features will be the images we use (in this case, a 
dog image or a cat image) and our labels will be a 0 or a 1. If the image is of a cat, the label will be 0. If the image is of a 
dog, the label will be a 1. If you were to add an additional class, its label would be 2. 

To get both the features and the labels, we created the following function to extract both from the image's file path. 


In [16]:
normpath = os.path.normpath('/notebooks/images/Cats/002_c0d2c2be.jpg')
split = normpath.split('/')

print(normpath)
print(split)
print(split[-2])

/notebooks/images/Cats/002_c0d2c2be.jpg
['', 'notebooks', 'images', 'Cats', '002_c0d2c2be.jpg']
Cats


In get_label, we take a file path (example is /notebooks/images/Cats/002_c0d2c2be.jpg), split it into a list containing
[notebooks, images, Cats, 002_c0d2c2be.jpg], and accessed the 2nd to last element where 'Cats' is. This 2nd to last element will 
always be the folder containing the images of the different classes (in this case Cats or Dogs, if you were to do more it could
be Birds or Tiger).

In process_file, we load the image from the file path (before we had the location of the image in the computer's memory, now we 
have the actual image), we then resize the image to 224 pixels by 224 pixels (this helps with model performance, CNNs struggle 
with images that are too big). The images are then converted to numpy arrays (this is for easier data conversion) and then 
normalized between 0 and 1. This is a common process in machine learning. Models perform better when the data is scaled to the
interval [0,1] or [-1,1]. In RGB images, the range of values is [0,255] when this is divided by 255, the new range is [0, 1]. 

In [5]:
CLASSES = ['Cats', 'Dogs'] # same name as the folder which hold the images

def get_label(file_path):
    label = os.path.normpath(file_path).split(os.path.sep)[-2] # extract the class from the label
    return CLASSES.index(label) # return the label 

def process_file(file_path, img_size=224):
    img = Image.open(file_path) # load the image from the url
    img = img.resize((img_size, img_size), Image.Resampling.BILINEAR) # resize the image to 224 x 224 pixels
    img = np.asarray(img) # convert Image to np array
    img = img/255.0 # scale image between 0 and 1 to improve model performance
    return img # return the image 


3. Create the datast

Now that we have the functionality to turn our image_urls to features and labels, its time to start constructed the dataset. 
The function below takes in a list which holds multiple classes of image urls. It will add all of these classes to the overall
training/testing features and labels. 

In [6]:
def create_dataset(datasets, training_proportion=0.8): # takes in list of image_urls ex. [cat_images, dog_images]
    
    training_features, training_labels = [], []
    testing_features, testing_labels = [], []
    
    for image_urls in datasets: # extract image_urls of a single class ex. cat_images
        for index, url in enumerate(image_urls): # loop through every url in cat_images

            if index < (len(image_urls) * training_proportion): # send a certain proportion of images for training, and the rest for testing

                training_features.append(process_file(url))
                training_labels.append(get_label(url))

            else: 
                testing_features.append(process_file(url))
                testing_labels.append(get_label(url))
                
                
    # these lists need to be converted to numpy arrays so the data conversion works, but essentially we're 
    # just returning the training_features, training_labels, testing_features, and testing_labels
    # we've added to above. They hold the same content. 
    return np.asarray(training_features).astype('float32'), np.asarray(training_labels).astype('float32'), 
           np.asarray(testing_features).astype('float32'), np.asarray(testing_labels).astype('float32')

Let's use this function!

In [7]:
training_features, training_labels, testing_features, testing_labels = create_dataset([cat_images, dog_images])

In [10]:
train_dataset = tf.data.Dataset.from_tensor_slices((training_features, training_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((testing_features, testing_labels))


2022-12-31 05:27:24.180430: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 96337920 exceeds 10% of free system memory.


4. Create the model

Now that we have our dataset created, it's time to create our model so we can train it! We're going to be using a convolutional 
neural network to analyze our images. In the previous chapter, we used a multilayered perceptron model. The issue with that
is it can only handle 1 dimensional data... but our images aren't 1 dimensional. The same trick we used last time (flattening the
images) won't work this time because these images are much more complex. Instead, we need our model to be able to actually
use 2D images without destroying spatial continuity (how close pixels are together, this is destroyed when flattening images). 

In [21]:
model = Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2))

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_dataset.batch(16), epochs=10, 
                    validation_data=test_dataset.batch(16))

Epoch 1/10
10/10 [==============================] - 30s 3s/step - loss: 15.9645 - accuracy: 0.5000 - val_loss: 1.6429 - val_accuracy: 0.5000
Epoch 2/10
10/10 [==============================] - 29s 3s/step - loss: 1.3036 - accuracy: 0.2000 - val_loss: 1.0915 - val_accuracy: 0.5000
Epoch 3/10
10/10 [==============================] - 29s 3s/step - loss: 1.0491 - accuracy: 0.2000 - val_loss: 0.7175 - val_accuracy: 0.5000
Epoch 4/10
10/10 [==============================] - 29s 3s/step - loss: 0.7417 - accuracy: 0.1688 - val_loss: 0.6927 - val_accuracy: 0.6000
Epoch 5/10
10/10 [==============================] - 29s 3s/step - loss: 0.6941 - accuracy: 0.3750 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 6/10
10/10 [==============================] - 29s 3s/step - loss: 0.6934 - accuracy: 0.5000 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 7/10
10/10 [==============================] - 29s 3s/step - loss: 0.6933 - accuracy: 0.5000 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 8/10
10/10 [=